In [1]:
import pandas as pd
import numpy as np
import nilearn
from nilearn.masking import apply_mask
import nibabel as nib
from glob import glob
from tqdm import tqdm
import os
from scipy.stats import zscore

In [2]:
base_dir = './data/'
con = ['adt', 'lh', 'rh', 'vis']

### extract voxel value

In [4]:
mask_nii = nib.load(base_dir + 'allSub_mask.nii')
mask_array = mask_nii.get_fdata()
mask_array = np.where(mask_array >= 0.5, True, 0)

for c in con:
    save_dir = base_dir + '_voxel_value/' + c + '/'
    os.makedirs(save_dir, exist_ok = True)
    for i in tqdm(range(1, 16), desc = c):
        data = nib.load(base_dir + 'sbj' + str(i).zfill(2) + '/' + c + '/swra' + c + '.nii')

        masked_img = apply_mask(data, mask_nii)

        df = pd.DataFrame(masked_img).T
        df_dir = save_dir + 'all_voxel_value_sub' + str(i).zfill(2) + '.xlsx'
        df.to_excel(df_dir)

vis: 100%|██████████| 15/15 [11:03<00:00, 44.25s/it]


### z-score normalization

In [6]:
for c in con:
    raw_voxel_dirs = glob(base_dir + '_voxel_value/' + c + '/all*.xlsx')

    for i, dir in enumerate(tqdm(raw_voxel_dirs, desc = c), start=1):
        df = pd.read_excel(dir, index_col = 'Unnamed: 0')
        zscored_df = zscore(df)

        save_dir = base_dir + '_voxel_value/' + c + '/zscored_all_voxel_value_sub' + str(i).zfill(2) + '.xlsx'
        zscored_df.to_excel(save_dir)

vis: 100%|██████████| 15/15 [1:35:50<00:00, 383.39s/it]  


### data segmentation

In [22]:
slice_num = []
for i in range(10):
    for j in [15, 35, 55]:
        slice_num.append(i + j)

slice_num = sorted(slice_num)

In [25]:
for c in con:
    norm_voxel_dirs = glob(base_dir + '_voxel_value/' + c + '/zscored_all*.xlsx')

    for i, dir in enumerate(tqdm(norm_voxel_dirs, desc = c), start=1):
        df = pd.read_excel(dir, index_col = 'Unnamed: 0')
        trimmed_df = df[slice_num]

        save_dir = base_dir + '_voxel_value/' + c + '/trimmed_zscored_all_voxel_value_sub' + str(i).zfill(2) + '.xlsx'
        trimmed_df.to_excel(save_dir)

vis: 100%|██████████| 15/15 [11:00<00:00, 44.06s/it]
